# Train Image Classifier with PyTorch on GPU Dask Cluster

Before you begin, make sure you have completed the steps in the [setup-external-connection.ipynb](setup-external-connection.ipynb) notebook.


In this project, we use the Stanford Dogs dataset, and starting with a pre-trained version of Resnet50, we will use transfer learning to make it perform better at dog image identification.

In [ ]:
import numpy as np
import os
import math
import datetime
import json 
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler
from dask_pytorch_ddp import data, dispatch
import multiprocessing as mp
from distributed.worker import logger # required for logging in Dask

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from torch.nn.parallel import DistributedDataParallel as DDP
from dask_saturn import SaturnCluster
from dask.distributed import Client
import torch.distributed as dist

## Set Model Specifications

Here you can assign your model hyperparameters, as well as identifying where the training data is housed on S3.

In [ ]:
model_params = {'n_epochs': 6, 
    'batch_size': 100,
    'base_lr': .01,
    'downsample_to':.5, # Value represents percent of training data you want to use
    'bucket': "saturn-public-data",
    'prefix': "dogs/Images",
    'pretrained_classes':imagenetclasses} 

### Label Formatting 
These utilities ensure the training data labels correspond to the pretrained model's label expectations.

In [ ]:
import re
import s3fs

##### Load label dataset
s3 = s3fs.S3FileSystem(anon=True)
with s3.open('s3://saturn-public-data/dogs/imagenet1000_clsidx_to_labels.txt') as f:
    imagenetclasses = [line.strip() for line in f.readlines()]
    
##### Format labels to match pretrained Resnet
def replace_label(dataset_label, model_labels):
    label_string = re.search('n[0-9]+-([^/]+)', dataset_label).group(1)
    
    for i in model_labels:
        i = str(i).replace('{', '').replace('}', '')
        model_label_str = re.search('''b["'][0-9]+: ["']([^\/]+)["'],["']''', str(i))
        model_label_idx = re.search('''b["']([0-9]+):''', str(i)).group(1)
        
        if re.search(str(label_string).replace('_', ' '), str(model_label_str).replace('_', ' ')):
            return i, model_label_idx
            break

## Training Function

This function encompasses the training task. 
* Load model and wrap it in PyTorch's Distributed Data Parallel function
* Set up DataLoader to iterate over training data
* Perform training tasks

In [ ]:
def simple_train_cluster(bucket, prefix, batch_size, downsample_to, n_epochs, base_lr, pretrained_classes):
    worker_rank = int(dist.get_rank())
    
    # --------- Format params --------- #
    device = torch.device("cuda")
    net = models.resnet50(pretrained=True) # True means we start with the imagenet version
    model = net.to(device)
    model = DDP(model)

    # --------- Set up eval --------- #
    criterion = nn.CrossEntropyLoss().cuda()    
    optimizer = optim.SGD(model.parameters(), lr=base_lr, momentum=0.9)

    # --------- Retrieve data for training --------- #
    transform = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(250), 
    transforms.ToTensor()])
    
    # Because we want to load our images directly and lazily from S3,
    # we use a custom Dataset class called S3ImageFolder.
    whole_dataset = data.S3ImageFolder(
        bucket, 
        prefix, 
        transform=transform, 
        anon = True
    )

    # Format target labels
    new_class_to_idx = {x: int(replace_label(x, pretrained_classes)[1]) for x in whole_dataset.classes}
    whole_dataset.class_to_idx = new_class_to_idx

    # ------ Create dataloader ------- #
    train_loader = torch.utils.data.DataLoader(
        whole_dataset, 
        sampler=RandomSampler(
            whole_dataset, 
            replacement = True,
            num_samples = math.floor(len(whole_dataset)*downsample_to)), 
        batch_size=batch_size, 
        num_workers=40, 
        multiprocessing_context=mp.get_context('fork'))   

    # --------- Start Training ------- #
    for epoch in range(n_epochs):
        count = 0
        model.train()
        
        for inputs, labels in train_loader:
            dt = datetime.datetime.now().isoformat()
            inputs, labels = inputs.to(device), labels.to(device)

            # Run model iteration
            outputs = model(inputs)

            # Format results
            _, preds = torch.max(outputs, 1)
            perct = [torch.nn.functional.softmax(el, dim=0)[i].item() for i, el in zip(preds, outputs)]
            
            loss = criterion(outputs, labels)
            correct = (preds == labels).sum().item()
            
            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            count += 1
                
            # Record the results of this model iteration for later review.
            if worker_rank == 0:
                logger.info(f"loss: {loss.item()}, correct: {correct}, epoch: {epoch}, count: {count}")


## Run Model

To run the model, we use the `dask-pytorch-ddp` function `dispatch.run()`. This takes our client, our training function, and our dictionary of model parameters. You can monitor the model run on all workers using the Dask dashboard.

Model performance statistics will print to the logs for Worker 0 inside the Saturn Cloud UI, but other model performance monitoring such as Weights and Biases or Tensorboard can be used instead.

In [ ]:
client.restart() # Clears memory on cluster- optional but recommended.

In [ ]:
%%time    
futures = dispatch.run(client, simple_train_cluster, **model_params)
futures

In [ ]:
# If one or more worker jobs errors, this will describe the issue
# futures[0].result()

## Housekeeping

Because we are not working inside the UI, we want to make sure that we close down any resources when we are done- otherwise undesired costs can be incurred.

To shut down the cluster entirely:

In [ ]:
client.close()
cluster.close()